## RL Project
This Jupyter notebook is a base for importing the [Arcade Learning Environment](https://ale.farama.org/getting-started/#) (ALE) which is part of the [Gymnasium](https://gymnasium.farama.org) project. ALE can be used to create an environment for various Atari 2600 games for training reinforcement learning models.


In [1]:
!pip install ale-py
!pip install "gymnasium[other]"
# !pip install --user --upgrade git+http://github.com/pyglet/pyglet@pyglet-1.5-maintenance

## Import the native ALE interface as ale_py
Taken from <https://ale.farama.org/getting-started/>

In [1]:
from ale_py import ALEInterface
ale = ALEInterface()

A.L.E: Arcade Learning Environment (version 0.11.2+ecc1138)
[Powered by Stella]


## Inline rendering example

In [3]:
# Renders some sample frames within the Jupyter notebook. 
# It's not very fast!

import matplotlib.pyplot as plt
import gymnasium as gym
from IPython import display
%matplotlib inline

# Initialise the environment
env = gym.make('ALE/Breakout-v5', render_mode='rgb_array')

# Reset the environment to generate the first observation
observation, info = env.reset(seed=42)

for step in range(100):
    # Render the environment  
    plt.imshow(env.render())
    display.display(plt.gcf())
    display.clear_output(wait=True)
    
    # Choose a random action, this is where we would insert a policy
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    
    # If the episode has ended then we can reset to start a new episode
    if terminated or truncated:
        observation, info = env.reset()

env.close()

In [4]:
# Renders some sample frames in a separate window. 
# Smoother than in inline rendering but closing the window resets the Jupyter notebook kernal

import gymnasium as gym
import ale_py

gym.register_envs(ale_py)

# Initialise the environment
env = gym.make("ALE/Breakout-v5", render_mode="human")

# Reset the environment to generate the first observation
observation, info = env.reset(seed=42)
for _ in range(1000):
    # Choose a random action, this is where we would insert a policy
    action = env.action_space.sample()

    # step (transition) through the environment with the action
    # receiving the next observation, reward and if the episode has terminated or truncated
    observation, reward, terminated, truncated, info = env.step(action)

    # If the episode has ended then we can reset to start a new episode
    if terminated or truncated:
        observation, info = env.reset()

env.close()

In [3]:
# Pseudo code for Deep Q-learning with Experience Replay (from the course notes)

# Initialise replay memory 𝐷 to capacity 𝑁
# Initialise action-value network 𝑞̂1 with parameters 𝜽1 ∈ ℝ𝑑 arbitrarily 
# Initialise target action-value network 𝑞̂2 with parameters 𝜽2 = 𝜽1

# Loop for each episode:  
    # Initialise 𝑆 

    # Loop for each step of episode:  
        # Choose action 𝐴 in state 𝑆 using policy derived from 𝑞̂1(𝑆,⋅,𝜃1) 
        # Take action 𝐴, observe reward 𝑅 and next-state 𝑆′ 
        # Store transition (𝑆,𝐴,𝑅,𝑆′) in 𝐷 
        # For each transition (𝑆𝑗,𝐴𝑗,𝑅𝑗,𝑆′𝑗) in minibatch sampled from 𝐷:
            # 𝑦={𝑅𝑗 if 𝑆′𝑗 is terminal, 𝑅𝑗+𝛾max𝑎′𝑞̂2(𝑆′𝑗,𝑎′,𝜽2) otherwise 
            # 𝑦̂=𝑞̂1(𝑆𝑗,𝐴𝑗,𝜽1) 
            # Perform gradient descent step ∇𝜃1𝐿𝛿(𝑦,𝑦̂) 

    # Every 𝐶 time-steps, update 𝜽2=𝜽1


In [2]:
# List of hyperparameters and their values (from the Google DeepMind paper)

minibatch_size          = 32          # Number of training cases over which each SGD update is computed.
replay_memory_size      = 1000000     # SGD udpates are sampled from this number of most recent frames.
agent_history_length    = 4           # The number of most recent frames that are given to the Q‑network.
target_update_freq      = 10000       # The frequency with which the target network is updated.
discount_factor         = 0.99        # Discount factor γ used in Q‑learning update.
action_repeat           = 4           # Repeat each selected action this many times.
update_frequency        = 4           # The number of actions selected by the agent between SGD updates.
learning_rate           = 2.5e-4      # The learning rate used by RMSProp.
gradient_momentum       = 0.95        # Gradient momentum used by RMSProp.
squared_grad_momentum   = 0.95        # Squared‑gradient (denominator) momentum used by RMSProp.
min_squared_gradient    = 0.01        # Constant added to squared gradient in the denominator of the RMSProp update.
initial_exploration     = 1.0         # Initial ε in ε‑greedy exploration.
final_exploration       = 0.1         # Final value of ε in ε‑greedy exploration.
final_exploration_frame = 1000000     # Number of frames over which the initial value of ε is linearly annealed to its final value.
replay_start_size       = 50000       # A uniform random policy is run for this number of frames before learning starts and is used to populate the replay memory.
noop_max                = 30          # Max number of "do nothing" actions performed by the agent at the start of an episode.
